# **深度学习公开课 - 深度学习中的时间序列算法群**
> 节选自《深度学习实战》第7期正课<br>
> 作者：@菜菜TsaiTsai<br>
> 版本号：2023/10/25<br>

## **<center><font color ="k">公开课Day3 时序算法进阶：时序卷积网络TCN<br><br>极致易学 | 高效入门 | 前景讨论<center>**<br>**<center><font color ="red">直播将于8点35分正式开始！<br><br>扫码回复"DL999"领取今日直播课件>>><br><br>扫码回复"优惠"抢直播间专属优惠券>>></font></center>**

## 0 课程规划

欢迎来到《深度学习中的时间序列算法群》公开课。在这门课程中，我将带你从0认识5大类深度学习中的时间序列模型，并为你深度讲解深度时序算法众多的的精彩理念与实现方式。当你完成这门课程时，你将完成深度时序算法入门，打好进一步学习更多高级架构的基础。

**DAY 1：LSTM与深度学习中的时间序列**
1. 深度学习中的时间序列数据
2. 时序数据 vs 非时序数据
3. 循环神经网络如何处理时序问题
4. LSTM的灵感起源与直觉理解
5. LSTM的基本结构与架构设计

**DAY 2：LSTM的参数全解与预测实战**
1. PyTorch中的LSTM层与参数
2. LSTM类的输入与输出
3. 股价与时间序列数据预测实战

**DAY 3：时序进阶：时序卷积网络TCN**
1. 1d卷积操作与时序数据
2. 因果卷积 Causal Convolution
3. 1d膨胀卷积与感受野扩张
4. 时序卷积网络的架构与实现
5. 深度学习中的5大类时序解决方案

**DAY 4：Transformer与Attention用于时间序列**

**DAY 5：Informer架构解析与时序应用**

**DAY 6：深度时序SOTA架构TabNet**

更多后续课程请关注B站动态和小可爱私聊信息！

## 1 从循环网络到卷积网络

- **循环网络家族的问题**

在前面的课程中，我们详细了解了深度学习中的时间序列数据以及循环神经网络家族算法，我们深度地解析了RNN、LSTM，并了解了循环网络处理时间序列问题的基本思路。我们了解到，时间序列数据之所以特别，是因为其信息量不止蕴含在特征之中，也蕴含在样本与样本之间。因此**在处理时间序列数据时，我们不仅要学习特征与标签之间的关系，还需要学习样本与样本之间的关系**。对时间序列数据而言，样本与样本之间的关系就是上一个时间点与下一个时间点之间的关系，因此**循环神经网络家族采取的手段是——依次处理时间点、并将上一个时间点的信息传递融入下一个时间点的信息的运算过程，使得下一个时间点能够获得上一个时间点传来的信息，从而在两个时间点中建立联系**。这是早年深度学习算法在处理时间序列数据时的一般思路。这一思路确实很巧妙，但是也存在着大量的问题——

- **运算效率过低**。必须依次处理时间点意味着计算无法并行，因此RNN和LSTM的运算效率有极大的问题，在数据量巨大的深度学习领域，计算效率低下是致命的缺陷之一。

- **简单的循环网络记忆机制不完善，有时遗忘太多，有时记住太多，难以处理长序列**。虽然循环网络家族建立起了时间点之间的联系，但这种联系有时不总是有效的，当一个时间点上包含的历史时间点信息越多，那些久远的历史时间点的信息就不太容易对当下的预测构成影响，比如RNN在遗忘方面的缺陷是十分显著的，浅层的lstm也容易在记忆上出现问题。

- **复杂的循环网络虽然擅长记忆，但是运算流程过于复杂，参数量很大**。LSTM尝试在RNN的基础上将记忆机制改进，整体表现非常不错，但是LSTM的整体复杂程度却非常高，具有相同尺寸隐藏层的RNN的参数量只有LSTM的1/5不到。

- **训练难，冲突问题和梯度问题难以被根治**。权重冲突、长短期记忆的冲突、梯度消失、梯度爆炸问题都是RNN和LSTM常见的训练问题，这些问题阻碍了循环网络在复杂任务上获得较好的表现。

尽管循环神经网络已经有效地建立了时间点与时间点之间的关联，但他们的去诶按也十分明显。从1997年LSTM诞生以来，深度学习研究者依然在不断地寻找更为高效和创新的方法来处理这种时间序列数据。在2023年的今天，有大量精彩纷呈的时序数据处理思路以及算法，他们构建了深度学习时间序列算法群，其中非常引人注目的一脉算法就是卷积神经网络。

卷积神经网络通常被认为是计算机视觉领域的核心算法，但事实上在今天，卷积已经被拓展到语言、时序、生成式算法等各个领域。得益于卷积独特的计算方式，卷积神经网络可以轻松处理较长的序列，而不会令算法差生遗忘问题，同时由于卷积可以并行、卷积层的参数更少，因此卷积网络在处理时序数据时，效率更高、更轻量，在许多时候也可以获得超越循环网络算法群的表现。今天，我们将深入挖掘卷积网络在处理时间序列数据方面的能力，并探讨其背后的原因。希望这次的探讨能够为大家带来新的启示，让我们更好地理解时序卷积网络TCN的魅力。

## 2 卷积运算流程与基本运行逻辑

在了解卷积网络是如何处理时间序列之前，我们首先要了解如下如下事实——**卷积神经网络是使用多个卷积层构成的神经网络，卷积层是使用卷积运算的神经网络层**，在深度学习中，**卷积运算可以被理解成是一种按顺序对矩阵或序列进行点积的运算，而点积就是将矩阵/序列中的对应位置元素相乘相加、最终得到一个标量的运算**。让我们从点积开始理解。

点积是两个尺寸一致、维度一致的向量或矩阵，进行对应位置元素相乘相加、最终得到一个标量的运算，来看下面的两个矩阵：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/21OctOpenClass/23.PNG)

一个是字母a-i（字母矩阵），另一个是数字1-9（数字矩阵），尺寸都是3x3。现在，我们求解两个矩阵的点积，则可以得到：

**<center>点积 = $\boldsymbol{ a + 2b + 3c + 4d + 5e + 6f + 7g + 8h + 9i}$</center>**

对于两个序列，我们也可以计算点积，例如对序列[1, 2, 3]和序列[A,B,C]计算点积，则有：

![01](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/01.png)

**<center>点积 = $\boldsymbol{ a + 2b + 3c}$</center>**

卷积是按顺序对矩阵或序列进行点积的运算，那具体如何按顺序进行点积？虽然点积只能发生在尺寸维度相同的数据之间，但卷积却可以发生在尺寸不同的数据之间。如下所示，假设我们现在有一个10x2的矩阵和一个3x2的矩阵，我们可以让较小的矩阵让分别与大矩阵中的1-3行样本、2-4行样本、3-5行样本……直到8-10行样本做点积，仿佛在使用小矩阵**扫描**大矩阵一般，生成一串点积的结果。这样**从上至下、依据样本的顺序对矩阵进行扫描点积、并生成一个新序列的计算方式，就是一维卷积运算**。

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/05.png)

![03](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/06.png)

同样的，假设我们现在依然拥有大矩阵（尺寸为10x3），一个小矩阵（尺寸为2x2）。我们依然让小矩阵“扫描”大矩阵，但此时，我们并不只是从上至下进行扫描，而是让小矩阵从左到右、再从上到下进行扫描——图图所示，小矩阵分别与大矩阵中的1、2、11、12号样本、11、12、21、22号样本、2、3、12、13好号样本……直到19、20、29、30号样本做点积，生成一组点积的结果。这样**从左到右、同时从上至下进行点积、并生成一个新矩阵的计算方式，就是二维卷积运算**。

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/07.png)

![03](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/08.png)

很显然，一维卷积只按照样本顺序进行扫描，而二维卷积会按照特征的顺序、样本的顺序同时进行扫描。**扫描方向的不同，是一维卷积与二维卷积的根本区别**，由于扫描方向不同，**因此一维卷积只会生成序列，而二维卷积往往生成矩阵**。

如果将上述流程类比到深度学习当中，那大矩阵就是我们需要处理的原始数据，小矩阵就是卷积神经网络根据算法使用者的需求生成的、专用于与原始数据进行点积运算的**卷积核**。卷积核的**长和宽**往往可以由算法使用者自定义，只不过在一维卷积当中，算法使用者只能定义卷积核的长度，一维卷积核的宽度一定是与原始数据的特征数量一致的；在二维卷积中，算法使用者既可以定义卷积核的长度、又可以定义卷积核的宽度，一般来说我们使用的是正方形的卷积核。

卷积核的尺寸是算法使用者定义的，那卷积核中的数字从哪里来呢？事实上，这些数字是在神经网络实例化时随机生成的——你或许已经想到了，**卷积核中的数字就是卷积神经网络的权重**$w$，卷积神经网络的训练和迭代流程就是找到最适合当前原始数据的卷积核的流程。卷积运算的过程，本质就是特征提取、信息提炼的过程，因此只要找到适当的卷积核，就可以对信息进行最为有效的提取，从而能够进行有效的预测。

**以上的流程可以完美地对应到卷积层当中**，大矩阵就是卷积层的输入，点积序列和矩阵就是卷积层的输出，而卷积核就是卷积层的权重。所以卷积层是输入矩阵，输出序列/矩阵的神经网络层。

- **卷积运算与时间序列有什么联系？卷积为什么很适合时间序列？**

现在我们已经知道卷积层是如何运作的了，但是这与时间序列有什么关系呢？在深度学习的世界中，一维卷积和二维卷积都可以被用于时间序列的处理，不过一维卷积用于时间序列的时候更多，我们来看看具体如何操作——在时间序列的数据展示方式中，最为简单的就是单变量时序数据，这是只有时间点作为索引、没有特征、只有标签的数据类型——

|时间|股价|
|:-:|:-:|
|9：00|xxx|
|9：01|xxx|
|9：02|xxx|
|9：03|xxx|
|9：04|xxx|
|9：05|xxx|
|……||
|9：56|xxx|
|9：57|xxx|
|9：58|xxx|

或者——

|日期|摄氏度|
|:-:|:-:|
|6月1日|xxx|
|6月2日|xxx|
|6月3日|xxx|
|6月4日|xxx|
|6月5日|xxx|
|6月6日|xxx|
|……||
|6月28日|xxx|
|6月29日|xxx|
|6月30日|xxx|

单变量时间序列就是一列从上向下按时间顺序排列的数字。我们可以在这串数字上进行一维卷积运算，以提取原始时间序列的信息、生成新的序列。此时，由于输入的数据是一维的，所以卷积核也是一维的（宽为1），我们将可以按照如下的方式处理时间序列数据：

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/02.png)

在深度学习当中，大部分的序列数据是多变量的——

|时间|开盘价|收盘价|交易量|……|波动率|
|:-:|:-:|:-:|:-:|:-:|:-:|
|9：00|xxx|xxx|xxx|xxx|xxx|
|9：01|xxx|xxx|xxx|xxx|xxx|
|9：02|xxx|xxx|xxx|xxx|xxx|
|9：03|xxx|xxx|xxx|xxx|xxx|
|9：04|xxx|xxx|xxx|xxx|xxx|
|9：05|xxx|xxx|xxx|xxx|xxx|
|……||||||
|9：56|xxx|xxx|xxx|xxx|xxx|
|9：57|xxx|xxx|xxx|xxx|xxx|
|9：58|xxx|xxx|xxx|xxx|xxx|

那此时我们可以使用宽度=特征数量的卷积核来对所有的特征进行并行处理。如下图所示，面对3个特征的时间序列/文字序列矩阵，我们可以使用结构为（自定义长度,3）的卷积核对序列进行扫描。此时，**一个卷积核会覆盖到的数据范围不止跨越时间点，还跨越特征值，因此卷积可以同时捕获到样本与样本之间、特征与样本之间、特征与特征之间、以及所有信息与标签之间的联系**。这是一种效率极高的方式，通过并行的运算，就可以一次性捕获所有相关的信息。

![04](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/04.gif)

当然，面对更高维度的时间序列数据，例如最常见的三维时间序列——

![](http://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/Live/NLP%26LLMs/03_.png)

卷积不止能够在特征上并行，还能够同时对多张时间序列表单进行处理，假设现在有红、绿、蓝三张时间序列表单，则我们可以一次性扫描掉所有表单的数据——

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/09.gif)

因此，面对结构为(batch_size, time_step, input_dimension)的三维时间序列数据，卷积可以一次性扫描掉这个batch中所有的表单，从而达到超高效并行的目的。现在你知道卷积网络是如何在时间序列数据上运行的了，**那卷积网络在时间序列上运行，有何独特优势呢**？

> **1. 丰富的并行机制，大幅提升模型的效率**

在循环神经网络中，我们必须依次对样本进行正向传播，但是在卷积网络中，我们可以一次性对多个样本进行卷积。最夸张的情况下，我们可以使卷积核的长度就等同于时间步的数量，一次卷积运算即可获得全部样本上的信息，这个效率是大幅提升。除此之外，卷积可以在不同的时间序列表单上并行，这使得卷积网络的效率天生会高于循环类网络。

> **2. 可以对同一套数据进行不同的解读**

在卷积网络中，一次扫描就相当于一次对信息的处理和解读，**我们可以创建多个卷积核、对同一组数据进行多次的扫描**，以逼迫算法以不同的方式来解读同样的数据——

![](http://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/12.png)

有几个卷积核、进行几次扫描，就可以在一维卷积中生成几个序列。所以一维卷积的输出究竟有多少列，是由人为规定的。对同样的数据进行不同的解读，有助于算法从不同的角度去提取信息，并且幸运的是，**使用不同的卷积核对数据进行扫描这一事项也是可以并行的**，因此增加扫描的次数并不会大幅降低卷积神经网络的运算效率。

> **3. 卷积天然可以获取多个样本上的信息**

卷积运算可以一次性扫描到多个时间点，因此卷积运算天生就能够非常巧妙地提取到时间点与时间点之间的联系，因为**当一个卷积核进行卷积运算后，新生成的点积序列中的每个值都会包含不止一个时间点的信息**，这就让卷积运算天生拥有“整合不同时间点之间信息”的能力，正适合于时间序列预测中“必须探索时间点与时间点之间联系”的根本需求。

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/13.png)

但除此之外，卷积网络可以通过叠加卷积层的方式，让一个点积结果中包含非常长的序列上的信息。当一个卷积层输出了点积矩阵后，这个点积矩阵可以作为下一个卷积层的输入，我们可以重新生成卷积核、在上个卷积层生成的点击矩阵上进行扫描。当第二个卷积核完成运算，输出心的点积矩阵后，这个
点积矩阵上的样本将大幅包含原始序列上的信息，如下图所示——

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/14.png)

一个点积矩阵上的值能够映射到原始矩阵上的范围被称之为是“当前卷积层的感受野”，感受野越大，一个点积值能够包含的时间序列就越长。可以看到，卷积网络可以轻松通过堆叠卷积层的方式来让算法拥有“长时间、长距离视野”，从而让算法拥有能够处理长序列的能力。这一点是循环网络望其项背的。

由于卷积的数学流程可以轻松在不同时间点的信息之间建立联系，同时卷积还能够实现超高效的并行策略，因此卷积神经网络可以处理任意序列数据，但是要处理时间序列数据还需要更多的改进和修饰。在TCN当中，我们主要使用的是一维卷积，同时TCN的作者还对一维卷积进行了改进，让我们一起来看看TCN当中的卷积层究竟是什么样的。

## 3 TCN中的卷积层与基本元件

在TCN当中，最为核心的计算结构被称之为膨胀因果卷积，它由膨胀卷积和因果卷积两种卷积构成，我们来看一下——

- **因果卷积**

在深度学习和信号处理中，卷积是一个常见的操作。但在处理时间序列数据时，特别是在预测未来的时刻，我们不希望未来的信息在当前时刻被使用。但是在一维卷积的计算过程中，很显然我们会涉及到的不止过去的时间点，还有未来的时间点（即不止会涉及到上方的样本，还会涉及到下方的样本）——

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/02.png)

这就引入了所谓的“因果卷积”。因果卷积保证了在任何时间点t，输出只依赖于时间点t及其之前的输入，而不依赖于t之后的输入。因果卷积可以通过对输入数据进行适当的“填充”来实现。具体地说，假设我们有一个1D的输入序列和一个大小为k的卷积核，为了实现因果卷积，我们可以在序列的开始处填充k-1个零，然后进行标准的卷积操作。这样，卷积的输出在任何时间点t都只会依赖于时间点t及其之前的输入，如下图所示：

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/10.png)

- **膨胀卷积**

膨胀卷积（Dilated Convolution）是TCN（Temporal Convolutional Network）中的关键组件，它可以通过对卷积核填上“空洞”的方式来放大卷积层的感受野。填补空洞的方式是卷及操作中常见的方式，这种方式无需增加模型参数或计算成本，就可以轻松放大感受野，我们来看看具体是如何操作。

在标准的卷积中，卷积核的元素是连续的，一次覆盖输入数据的连续部分。而在膨胀卷积中，卷积核的元素之间存在间隔，这些间隔使得卷积核可以覆盖更广的范围。如下所示，当我们使用膨胀指数为1时，就是在原始卷积核的每行中填补一行0——

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/15.png)

膨胀卷积可以很大程度帮助我们放大感受野，当卷积层堆叠的时候——

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/16.png)

很显然，如果我们使用更大的膨胀指数，那感受野就可以被放得更大。在TCN当中，原作者建议的膨胀指数是第一个卷积层使用1，第二个卷积层使用2，第三个卷积层使用4，这种结构可以使网络的顶层捕捉到非常长的时间依赖关系，而底层则可以捕捉到更短的依赖关系。

- **膨胀因果卷积与残差链接**

残差连接（或残差块、残差链接）是深度学习中一种增强网络训练稳定性的技术。它首次由He等人在2015年的文章中提出，用于解决深层网络中的梯度消失和梯度爆炸问题。这种设计后来在多种网络架构中被广泛采用，包括TCN（Temporal Convolutional Network）。

在TCN中，残差链接的主要目的是帮助模型学习不同时间尺度上的依赖关系，并确保深度增加时的训练稳定性。在残差连接的设计中，当前层的输出不仅传递给下一层，而且与输入直接相加，从而形成一个“短路”连接。这种设计允许信息直接流过多个层，提供了一种更直接的路径更新梯度。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/1dcnn/17.png)

在TCN中，残差链接构建了残差块，而残差快由以下路径组成：

**主路径**：输入首先通过一系列操作，例如卷积、批量归一化和激活函数。

**残差路径**：输入不经过任何操作直接进行。

**融合**：主路径和残差路径的输出在深度方向上相加，形成最终的输出。

这种结构确保了，如果模型认为当前层的操作不会为最终的输出增加任何有益的信息，那么它可以将这些操作的权重设置得很小，从而使主路径的输出接近于零。这样，模型的输出就主要依赖于残差连接。这给予模型一种选择，要么使用主路径，要么依赖残差路径。在实际实现中，为了确保主路径的输出和残差路径的输出具有相同的形状，可能需要对输入或输出进行某种修改。例如，如果主路径中的卷积操作改变了特征的数量，那么可能需要在残差路径中添加一个1x1的卷积来匹配特征数量。

## 4 一维卷积的参数与TCN的实现

在之前的课程当中，我们已经认识了PyTorch框架的基本结构，整个PyTorch框架可以大致被分Torch和成熟AI领域两大板块，其中Torch包含各类神经网络组成元素、用于构建各类神经网络，各类AI领域中则包括Torchvision、Torchtext、Torchaudio等辅助完成图像、文字、语音方面各类任务的领域模块。

在PyTorch中，LSTM是属于“构建循环神经网络的元素”，而非“成熟神经网络”，因此LSTM是位于PyTorch.nn这个基本模块下。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/Live/NLP%26LLMs/24.png)

![](http://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/Live/NLP%26LLMs/25.png)

来看看位于torch.nn模块下的Conv1d层：

`torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)`

如果你熟悉计算机视觉，那你看向一维卷积的类时一定会感觉到非常熟悉。在PyTorch当中，一维卷积层和二维卷积层的参数是100%一致的，让我们来看看这些重要参数：

**in_channels**: 输入数据的通道数。例如，如果你的输入数据是一个具有多个特征的时间序列，那么in_channels就等于这些特征的数量。

**out_channels**: 输出数据的列数。这等于卷积运算执行的次数，也就是卷积核的数量。

**kernel_size**:  卷积核的大小。在一维卷积中，这是一个单一的整数，表示卷积核覆盖的连续数据点数。

**stride** (默认为 1):卷积操作时卷积核移动的步长。如果步长大于1，输出的尺寸会减小。

**padding** (默认为 0): 在输入数据的两侧加上的零的数量。填充可以帮助控制输出的尺寸，特别是当你希望输出尺寸与输入尺寸相同时。

**dilation** (默认为 1): 膨胀卷积中的膨胀指数。当膨胀率大于1时，卷积核中的元素之间会有间隔，从而增大了卷积的感受野。

In [1]:
from torch import nn
import torch

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

2.0.1+cu118
True


In [39]:
m = nn.Conv1d(in_channels=16, out_channels=1, kernel_size=3, padding=1)
#输入的维度是三维，注意在PyTorch中，实际的维度是（batch_size，input_dimension，time_step）
input = torch.randn(20, 16, 50)
output = m(input)

In [40]:
output.shape #输出的维度也是三维

torch.Size([20, 1, 50])

In [41]:
m.weight.shape #这就是卷积核的大小

torch.Size([1, 16, 3])

基于一维卷积，我们来实现TCN的基本结构：

In [45]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm

In [46]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        """
        其实这就是一个裁剪的模块，裁剪多出来的padding
        """
        return x[:, :, :-self.chomp_size].contiguous()

In [47]:
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        """
        相当于一个Residual block

        :param n_inputs: int, 输入通道数
        :param n_outputs: int, 输出通道数
        :param kernel_size: int, 卷积核尺寸
        :param stride: int, 步长，一般为1
        :param dilation: int, 膨胀系数
        :param padding: int, 填充系数
        :param dropout: float, dropout比率
        """
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        # 经过conv1，输出的size其实是(Batch, input_channel, seq_len + padding)
        self.chomp1 = Chomp1d(padding)  # 裁剪掉多出来的padding部分，维持输出时间步为seq_len
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)  #  裁剪掉多出来的padding部分，维持输出时间步为seq_len
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        """
        参数初始化

        :return:
        """
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        """
        :param x: size of (Batch, input_channel, seq_len)
        :return:
        """
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

In [44]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        """
        TCN，目前paper给出的TCN结构很好的支持每个时刻为一个数的情况，即sequence结构，
        对于每个时刻为一个向量这种一维结构，勉强可以把向量拆成若干该时刻的输入通道，
        对于每个时刻为一个矩阵或更高维图像的情况，就不太好办。

        :param num_inputs: int， 输入通道数
        :param num_channels: list，每层的hidden_channel数，例如[25,25,25,25]表示有4个隐层，每层hidden_channel数为25
        :param kernel_size: int, 卷积核尺寸
        :param dropout: float, drop_out比率
        """
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i   # 膨胀系数：1，2，4，8……
            in_channels = num_inputs if i == 0 else num_channels[i-1]  # 确定每一层的输入通道数
            out_channels = num_channels[i]  # 确定每一层的输出通道数
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        """
        输入x的结构不同于RNN，一般RNN的size为(Batch, seq_len, channels)或者(seq_len, Batch, channels)，
        这里把seq_len放在channels后面，把所有时间步的数据拼起来，当做Conv1d的输入尺寸，实现卷积跨时间步的操作，
        很巧妙的设计。
        
        :param x: size of (Batch, input_channel, seq_len)
        :return: size of (Batch, output_channel, seq_len)
        """
        return self.network(x)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/LSTM/%E8%AF%BE%E7%A8%8B%E5%AE%A3%E4%BC%A011.jpg)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/LSTM/%E8%AF%BE%E7%A8%8B%E5%AE%A3%E4%BC%A05.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/LSTM/%E8%AF%BE%E7%A8%8B%E5%AE%A3%E4%BC%A06.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/LSTM/%E8%AF%BE%E7%A8%8B%E5%AE%A3%E4%BC%A09.png)

|||
|:-:|:-:|
|![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/LSTM/%E8%AF%BE%E7%A8%8B%E5%AE%A3%E4%BC%A01.png)|![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/LSTM/%E8%AF%BE%E7%A8%8B%E5%AE%A3%E4%BC%A07.png)|

# 【公开课周五继续】